In [1]:
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('../train.csv')
df_raw = df.copy()
df_raw.head()
## value가 문자 값이지만 등급을 나타내는 feature들 (nan값은 대상이 없음을 의미한다.)
tonum = {"ExterQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "ExterCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "HeatingQC": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "KitchenQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "FireplaceQu": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "GarageQual": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "GarageCond": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "PoolQC": {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, np.nan : 0},
         "BsmtFinType1" : {'GLQ' : 6, 'ALQ' : 5, 'BLQ': 4,'Rec' : 3,'LwQ' : 2, 'Unf' : 1, np.nan : 0},
         "BsmtFinType2" : {'GLQ' : 6, 'ALQ' : 5, 'BLQ': 4,'Rec' : 3,'LwQ' : 2, 'Unf' : 1, np.nan : 0},
           }
df_raw = df_raw.replace(tonum)
## 그 외 nan값이 모르는 값이 아닌 대상이 없음을 의미 하는 feature들
fill_na_ls = [['GarageType',# NA : 차고 없음 (category)
 'MiscFeature',             # NA : 없음  (category) 기타 범주(엘레베이터, 테이스 코트) 기타 범주
 'Fence',                   # NA : 울타리 없음 (category)
 'GarageFinish',            # NA : 차고 없음 (category)
 'BsmtExposure',            # NA : 지하실 없음
 'Alley',                   # NA : No alley access
]]

for name in fill_na_ls:
    df_raw[name] = df_raw[name].fillna('N')



for name in fill_na_ls:
    df_raw[name] = df_raw[name].fillna('N')
# 그래도 남은 Null값들
total = df_raw.isnull().sum().sort_values(ascending=False)
percent = (df_raw.isnull().sum()/df_raw.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
pd.set_option("display.max_rows",None)
missing_data.head()
# Null 값 drop
pd.set_option("display.max_columns",None)
df_raw = df_raw.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_raw = df_raw.drop(df_raw.loc[df_raw['Electrical'].isnull()].index)
# y값에 로그
from scipy import stats
df_raw['SalePrice'] = np.log(df_raw['SalePrice'])
# 1층과 2층 넓이 합침
df_raw['TotFlrSF'] = df_raw['1stFlrSF'] + df_raw['2ndFlrSF']

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_all = df_raw.copy()#BsmtQual 지하실 높이 등급   TotalBsmtSF 지하실 면적  FireplaceQu 벽난로 품질(등급)   HeatingQC  난방 품질(등급)
model_all = sm.OLS.from_formula("SalePrice ~ scale(OverallQual) + scale(I(GrLivArea))+ scale(I(GrLivArea ** 2))+ scale(I(GrLivArea ** 3)) + scale(KitchenQual) + scale(GarageCars) + scale(TotalBsmtSF) + scale(TotFlrSF) + scale(BsmtQual) + scale(FireplaceQu) + scale(YearRemodAdd) + scale(HeatingQC)", data=df_all) 
 
result_all = model_all.fit()
print(result_all.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     685.0
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:15:15   Log-Likelihood:                 654.52
No. Observations:                1459   AIC:                            -1283.
Df Residuals:                    1446   BIC:                            -1214.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [3]:
# 아웃라이어 제거
influence = result_all.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (1460 - 1 - 13)
idx = np.where(cooks_d2 > fox_cr)[0]
df_fin = df_all.copy()
df_fin = df_fin.drop(df_fin.index[idx])

df_all = df_fin.copy()#BsmtQual 지하실 높이 등급   TotalBsmtSF 지하실 면적  FireplaceQu 벽난로 품질(등급)   HeatingQC  난방 품질(등급)
model_all = sm.OLS.from_formula("SalePrice ~ scale(OverallQual) + scale(I(GrLivArea))+ scale(I(GrLivArea ** 2))+ scale(I(GrLivArea ** 3)) + scale(KitchenQual) + scale(GarageCars) + scale(TotalBsmtSF) + scale(TotFlrSF) + scale(BsmtQual) + scale(FireplaceQu) + scale(YearRemodAdd) + scale(HeatingQC)", data=df_all) 
 
result_all = model_all.fit()
print(result_all.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     963.9
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:15:16   Log-Likelihood:                 979.60
No. Observations:                1377   AIC:                            -1933.
Df Residuals:                    1364   BIC:                            -1865.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [6]:
# 제출 결과 만들기

# test데이터 전처리
df_test = pd.read_csv('../test.csv')
df_test = df_test.replace(tonum)
for name in fill_na_ls:
    df_test[name] = df_test[name].fillna('N')
    
df_test['TotFlrSF'] = df_test['1stFlrSF'] + df_test['2ndFlrSF']
df_test = df_test.fillna(method='ffill')


pre = pd.DataFrame(result_all.predict(df_test))
pre['Id']=pre.index
pre = pre.rename(columns={0: 'SalePrice'})
pre['Id'] = range(1461,2920)
pre['SalePrice'] = pre['SalePrice'].apply(lambda x : np.exp(x))
pre.to_csv('submission.csv', index=False)

In [23]:
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from patsy import dmatrices
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

y, X = dmatrices("SalePrice ~ scale(OverallQual) + scale(I(GrLivArea))+ scale(I(GrLivArea ** 2))+ scale(I(GrLivArea ** 3)) + scale(KitchenQual) + scale(GarageCars) + scale(TotalBsmtSF) + scale(TotFlrSF) + scale(BsmtQual) + scale(FireplaceQu) + scale(YearRemodAdd) + scale(HeatingQC)", df_all, return_type = 'dataframe' )

#poly = PolynomialFeatures(1)
model1_LR = LinearRegression().fit(X, y)
model1_ridge = Ridge().fit(X, y)
model1_lasso = Lasso().fit(X, y)
model1_EN = ElasticNet().fit(X, y)
                
cv = KFold(10, shuffle=True)
#cross_val_score(model1_LR, X, y, scoring="r2", cv=cv)
print('[Original LR]')
print('score : ', cross_val_score(model1_LR, X, y, scoring="r2", cv=cv),'\n',
      'mean : ', np.mean(cross_val_score(model1_LR, X, y, scoring="r2", cv=cv)),'\n')

print('[Ridge]')
print('score : ', cross_val_score(model1_ridge, X, y, scoring="r2", cv=cv),'\n',
      'mean : ', np.mean(cross_val_score(model1_ridge, X, y, scoring="r2", cv=cv)),'\n')
      #'std : ', np.std(cross_val_score(model1_ridge, X, y, scoring="r2", cv=cv)))

print('\n[Lasso]')
print('score : ', cross_val_score(model1_lasso, X, y, scoring="r2", cv=cv),'\n',
      'mean : ', np.mean(cross_val_score(model1_lasso, X, y, scoring="r2", cv=cv)),'\n')
      #'std : ', np.std(cross_val_score(model1_lasso, X, y, scoring="r2", cv=cv)))

print('\n[Elastic-net]')
print('score : ', cross_val_score(model1_EN, X, y, scoring="r2", cv=cv),'\n',
      'mean : ', np.mean(cross_val_score(model1_EN, X, y, scoring="r2", cv=cv)),'\n')
#      'std : ', np.std(cross_val_score(model1_EN, X, y, scoring="r2", cv=cv)))

[Original LR]
score :  [0.90552329 0.85059752 0.91287087 0.88596957 0.89129777 0.88675355
 0.87597127 0.89641205 0.90025294 0.89962174] 
 mean :  0.8912620794998739 

[Ridge]
score :  [0.92558101 0.86570317 0.8684949  0.92088321 0.881722   0.89216156
 0.87704955 0.89612485 0.88783487 0.89526406] 
 mean :  0.8897030373801875 


[Lasso]
score :  [-0.00057038 -0.00410214 -0.02210123 -0.00528488 -0.00057459 -0.01471654
 -0.01469669 -0.01841302 -0.00023733 -0.0257072 ] 
 mean :  -0.004683419807497047 


[Elastic-net]
score :  [-0.01824085 -0.00068914 -0.01222933 -0.01049015 -0.00038456 -0.00368703
 -0.00437943 -0.00110213 -0.00022824 -0.0132855 ] 
 mean :  -0.011065697854732392 



In [22]:
result_all.

In [ ]:
from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
cv = KFold(10, shuffle=True)
y, X = dmatrices("Y ~ X", df, return_type = 'dataframe' )
model = LinearRegression().fit(X, y)
cross_val_score(model, X, y, scoring="r2", cv=cv)
